In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException

In [2]:
def cofepris(medicamento):
    #función para cambiar la denominación del medicamento
    def denominacion(option_text):
        combo = driver.find_element(By.CLASS_NAME, "form-control")
        combo_select = Select(combo)
        combo_select.select_by_visible_text(option_text)
        cuadro_busqueda = driver.find_element(By.ID, "MainContent_txtSearchValue")
        cuadro_busqueda.send_keys(medicamento)
        button = driver.find_element(By.XPATH, "//button[contains(@onclick, '__doPostBack') and @class='btn btn-primary']")
        button.click()
    #Función para extraer los elementos de la tabla
    def elementos_tabla(driver):
        table_element = driver.find_element(By.CLASS_NAME, "gridView_basic")
        table_contents = table_element.get_attribute('outerHTML')
        return table_contents
    
    #Código para extraer los datos de la página
    url = "https://tramiteselectronicos02.cofepris.gob.mx/BuscadorPublicoRegistrosSanitarios/BusquedaRegistroSanitario.aspx"
    options = webdriver.ChromeOptions()
    options.add_argument('--incognito')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--disable-cache')
    options.add_argument('--disable-cookies')
    driver = webdriver.Chrome(service=Service("C:/chromedriver.exe"), options=options)
    driver.get(url)
    #Denominación Distintiva
    denominacion("Denominación Distintiva")

    try:
        table_contents = elementos_tabla(driver)
    except NoSuchElementException:
        #Denominación Genérica
        denominacion("Denominación Genérica")
        try:
            table_contents = elementos_tabla(driver)
        except NoSuchElementException:
            driver.quit()
            return pd.DataFrame({'marca': [], 'generica': [], 'laboratorio': [], 'fabricante': []})

    soup = BeautifulSoup(table_contents, 'html.parser')
    marca = []
    generica = []
    lab = []
    fabricante = []

    for span in soup.find_all('span'):
        if span.has_attr('id'):
            if span['id'].startswith('MainContent_GV_RegistroSanitario_lblDenominacionDistintiva'):
                marca.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblDenominacionGenerica'):
                generica.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblTitularRegistro'):
                lab.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblFabricanteMedicamento'):
                fabricante.append(span.text)

    df = pd.DataFrame({'marca': marca, 'generica': generica, 'laboratorio': lab, 'fabricante': fabricante})
    df = df.apply(lambda x: x.astype(str).str.lower())
    df = df.apply(lambda x: x.astype(str).str.replace('\n', ''))
    driver.quit()
    return df


## Ejemplo

In [3]:
lista_meds=["histiacil","ibuprofeno"]

In [4]:
df=pd.DataFrame()
for med in lista_meds:
    df=pd.concat([df,cofepris(med)])

In [5]:
df

,marca,generica,laboratorio,fabricante
0,histiacil nf,dextrometorfano / ambroxol,"sanofi-aventis de méxico, s. a. de c. v.","gelcaps exportadora de méxico, s. a. de c. v"
1,histiacil natix,hedera hélix,sanofi-aventis de méxico s.a. de c.v.,"sanofi-aventis de méxico, s.a. de c.v."
2,histiacil flu,cafeína / fenilefrina / paracetamol,sanofi-aventis de méxico s.a. de c.v.,"sanofi-aventis de méxico, s.a. de c.v."
3,histiacil fam,guaifenesina/oxolamina,"sanofi aventis de mexico, s.a. de c.v.","sanofi-aventis de méxico, s.a. de c.v."
4,histiacil-fam,guaifenesina / oxolamina,"sanofi aventis de mexico, s.a. de c.v.","sanofi-aventis de méxico, s.a. de c.v."
0,arbufentab,ibuprofeno,vitae laboratorios s.a de c.v.,"gelpharma, s. a. de c. v."
1,adopren,ibuprofeno,"genetica laboratorios, sa de cv","genética laboratorios, s.a. de c.v."
2,brysbal,ibuprofeno / cafeína,"gelpharma, s.a de c.v.","gelpharma, s. a. de c. v."
3,pedea,ibuprofeno,,recordati industria chimica e farmaceutica s.p.a.
4,actron,ibuprofeno,bayer de mexico s.a. de c.v.,
